In [1]:
def get_nearby_point(origin):
    import geopy.distance
    from random import random
    dist = geopy.distance.distance(kilometers = .02)
    pt = dist.destination(point=geopy.Point(origin), bearing=random()*360)
    return pt[0],pt[1]

def zoom_center(lons: tuple=None, lats: tuple=None, lonlats: tuple=None,
        format: str='lonlat', projection: str='mercator',
        width_to_height: float=2.0) -> (float, dict):
    import numpy as np
    if lons is None and lats is None:
        if isinstance(lonlats, tuple):
            lons, lats = zip(*lonlats)
        else:
            raise ValueError(
                'Must pass lons & lats or lonlats'
            )
    
    maxlon, minlon = max(lons), min(lons)
    maxlat, minlat = max(lats), min(lats)
    center = {
        'lon': round((maxlon + minlon) / 2, 6),
        'lat': round((maxlat + minlat) / 2, 6)
    }
    
    # longitudinal range by zoom level (20 to 1)
    # in degrees, if centered at equator
    lon_zoom_range = np.array([
        0.0007, 0.0014, 0.003, 0.006, 0.012, 0.024, 0.048, 0.096,
        0.192, 0.3712, 0.768, 1.536, 3.072, 6.144, 11.8784, 23.7568,
        47.5136, 98.304, 190.0544, 360.0
    ])
    
    if projection == 'mercator':
        margin = 1.2
        height = (maxlat - minlat) * margin * width_to_height
        width = (maxlon - minlon) * margin
        lon_zoom = np.interp(width , lon_zoom_range, range(20, 0, -1))
        lat_zoom = np.interp(height, lon_zoom_range, range(20, 0, -1))
        zoom = round(min(lon_zoom, lat_zoom), 2)
    else:
        raise NotImplementedError(
            f'{projection} projection is not implemented'
        )
    
    return zoom, center
def get_centroid(coords_df):
    import math
    x = 0.0
    y = 0.0
    z = 0.0

    for i, coord in coords_df.iterrows():
        latitude = math.radians(coord.latitude)
        longitude = math.radians(coord.longitude)

        x += math.cos(latitude) * math.cos(longitude)
        y += math.cos(latitude) * math.sin(longitude)
        z += math.sin(latitude)

    total = len(coords_df)

    x = x / total
    y = y / total
    z = z / total

    central_longitude = math.atan2(y, x)
    central_square_root = math.sqrt(x * x + y * y)
    central_latitude = math.atan2(z, central_square_root)

    mean_location = {
        'latitude': math.degrees(central_latitude),
        'longitude': math.degrees(central_longitude)
        }
    return mean_location

def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import pandas as pd
    import plotly.express as px
#     import geopandas as gpd
    
    import plotly.graph_objects as go
    token = open("/home/jupyter/sneupane/MOODS/visualizations/.mapbox_token").read()
#     geo_df = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
    
    data_df=data_df.loc[data_df['Stressor'].notnull()]
#     data_df=data_df.loc[data_df['Location'].notnull()]
    user_df=data_df.loc[data_df["user"]==user_id]
    user_df=user_df.loc[(user_df.latitude <=90) & (user_df.latitude>=-90)]
    if len(user_df)>0:
#         user_df_locations=user_df.groupby("Location",as_index=False)["counter"].count().sort_values(["counter"],ascending=False)
#         max_location=user_df_locations["Location"].iloc[0]
#         max_location_lat=user_df.loc[user_df["Location"]==max_location]["latitude"].values.tolist()[0]
#         max_location_long=user_df.loc[user_df["Location"]==max_location]["longitude"].values.tolist()[0]
        mean_location=get_centroid(user_df)
        zoom, mean_location = zoom_center(
        lons=user_df["longitude"],
        lats=user_df["latitude"])
        df_location = user_df.loc[user_df["Location"].notnull()]
        df_location_not = user_df.loc[user_df["Location"].isnull()]
        labeled_location=[]
        for index , row in df_location.iterrows():
            labeled_location.append([row["Location"],row["latitude"],row["longitude"]])


        from haversine import haversine, Unit
        label_test=[]
        for index , row in df_location_not.iterrows():
            presumed_loc=""
            for i in labeled_location:
                labeled=set(i[1:])
                temp=set([row["latitude"],row["longitude"]])
                haver_dist=haversine(i[1:], temp,unit='m')
                if haver_dist <=30 :
                    presumed_loc=i[0]
                    break
                else:
        #             print(i[0])
                    presumed_loc="unknown"
            label_test.append(presumed_loc)

        df_location_not["Location"]=label_test
        frames=[df_location,df_location_not]
        
        user_df = pd.concat(frames)
        
        
        



        color_map={"Morning":"red","Afternoon":"blue","Evening":"green","Night":"yellow"}
        original_latitudes=user_df["latitude"].values
        original_longitudes=user_df["longitude"].values
        modified_latitudes=[]
        modified_longitudes=[]

        for i,j in zip(original_latitudes,original_longitudes):

            temp,temp1=get_nearby_point((i,j))
            modified_latitudes.append(temp)
            modified_longitudes.append(temp1)

        user_df.drop("latitude",axis=1,inplace = True)
        user_df.drop("longitude",axis=1,inplace = True)

        user_df = user_df.assign(latitude=modified_latitudes)
        user_df = user_df.assign(longitude=modified_longitudes)

        
    
        user_df['color'] = user_df['start_time_of_day'].map(color_map)
        user_df.loc[:,"size"]=5
#         print(user_df)
        mapboxt =open("/home/jupyter/sneupane/MOODS/visualizations/.mapbox_token").read().strip()

        
        temp1=user_df.loc[user_df["week"] == week_num]
        this_week_location=temp1["Location"].unique().tolist()

        temp2=user_df.loc[user_df["week"] != week_num]
        last_week_location=temp2["Location"].unique().tolist()

#         print(this_week_location,last_week_location)
        new_location_list=set(this_week_location)-set(last_week_location)

        prev_weeks_loc_df=user_df.loc[user_df["Location"].isin(last_week_location)]
        this_week_loc_df=user_df.loc[user_df["Location"].isin(new_location_list)]


        user_df=pd.concat([this_week_loc_df,prev_weeks_loc_df])
        
#         user_df_locations=user_df.groupby("Location",as_index=False)["counter"].count().sort_values(["counter"],ascending=False)
#         max_location=user_df_locations["Location"].iloc[0]
#         max_location_lat=user_df.loc[user_df["Location"]==max_location]["Latitude"].values.tolist()[0]
#         max_location_lon=user_df.loc[user_df["Location"]==max_location]["Longitude"].values.tolist()[0]
        
        user_df=user_df[["latitude","longitude","Location","Stressor","start_time_of_day","color"]]
#         print(user_df)
        fig = go.Figure(data=[go.Scattermapbox(
                lat=prev_weeks_loc_df["latitude"],
                lon=prev_weeks_loc_df["longitude"],
                mode='markers+text',
               hoverinfo="text",
#                 customdata=np.dstack((prev_weeks_loc_df["Stressor"] ,prev_weeks_loc_df["color"])),
            customdata=prev_weeks_loc_df[["latitude","longitude","Location","Stressor","start_time_of_day","color","day_full"]] ,
                text=prev_weeks_loc_df["Location"], 
            showlegend=True,
            textposition="top center",
                 marker=go.scattermapbox.Marker(symbol ='circle',size=10),
                name="Previous locations",
             textfont=dict(
                        family="sans serif",
                        size=20,
                 color="#00429d",
            ),
#                             color=prev_weeks_loc_df["color"].values)

            hovertemplate =
"<b>Stressor:%{customdata[3]}</b><br><b>Day:%{customdata[6]}</b><br><b>Time of Day:%{customdata[4]}</b>",)
            ,
               go.Scattermapbox(
                 lat=this_week_loc_df["latitude"],
                lon=this_week_loc_df["longitude"],
                mode='markers+text',
                   hoverinfo="text",
                   customdata=this_week_loc_df[["latitude","longitude","Location","Stressor","start_time_of_day","color","day_full"]],
                name="New Locations this week " + str(week_num), textposition="top center",
                   text=this_week_loc_df["Location"],
                   textfont=dict(
                        family="sans serif",
                        size=20,
                       color="#e9002c",
#                             color=prev_weeks_loc_df["color"].values
                   ),
                   hovertemplate =
                       "<b>Stressor:%{customdata[3]}</b><br><b>Day:%{customdata[6]}</b><br><b>Time of Day:%{customdata[4]}</b>",
                 marker=go.scattermapbox.Marker(symbol ='circle',size=10,opacity=1),
                     )])

        fig.update_layout(legend=dict(orientation="h", itemclick="toggleothers",
                          itemdoubleclick="toggle",font=dict(
            family="Courier",
            size=10,
            color="black"
        ),
                          yanchor="bottom",y=1.05,xanchor="right",x=0.5),autosize=True,showlegend=True,
                          mapbox = dict(center= dict(lat=mean_location["lat"], lon=mean_location["lon"]),            
                             accesstoken= mapboxt,
                             zoom=zoom,
                             style="streets"
                           ))




        
        fig.update_layout(hoverlabel=dict(
                                    bgcolor="white",
                                    font_size=16,
                                    font_family="Rockwell",
                                    bordercolor="Black"
                                )
                            )
        
        fig.update_layout(
                    autosize=True,

                    margin=dict(
                        l=30,
                        r=30,
                        b=10,
                        t=10,
                       
                    ),)
        data_summary_user=data_summary_user_current["Map View"]
#         a=["a","b","c"]
#         b=["d","e","f"]
        if len(new_location_list) > 0:
            data_summary_user=data_summary_user.replace("_a_", " ,".join(new_location_list))
        else:
            data_summary_user=data_summary_user.replace("_a_", "No new locations reported this week" )
        
#         data_summary_user=data_summary_user.replace("_b_", " ,".join(last_week_location))
        data_summary_user=data_summary_user.replace("week_num", str(week_num))
#         data_summary_user=data_summary_user.replace("_d_", str(week_num))
       
        return "Map View",fig,data_summary_user
    else:
        return "","",""